# setup & load data

In [1]:
import pandas as pd
import plotly.express as px
from datetime import datetime

In [2]:
sale_df = pd.read_csv(r"D:\Desktop\marwa\marwa\RealEstate_AnalysisProject\03_Cleaned Dataset\sales_clean.csv")
rent_df = pd.read_csv(r"D:\Desktop\marwa\marwa\RealEstate_AnalysisProject\03_Cleaned Dataset\rent_clean.csv")

In [3]:
sale_df["transaction_date"]= pd.to_datetime(sale_df["sale_date"])
sale_df["commission_payment_date"]= pd.to_datetime(sale_df["commission_payment_date"])
sale_df.rename(columns={"sale_amount":"transaction_amount"}, inplace=True)

In [4]:
rent_df["transaction_date"]= pd.to_datetime(rent_df["agreement_date"])
rent_df["commission_payment_date"]= pd.to_datetime(rent_df["commission_payment_date"])
rent_df["transaction_amount"] = (rent_df["rent_amount"]*rent_df["rent_months"])

In [5]:
rent_df['transaction_type'] = 'rent'
sale_df['transaction_type'] = 'sale'
common_cols = ['property_id','agent_id','owner_id','client_id','transaction_date',"transaction_amount",'commission_payment_date','commission_amount','transaction_type']
transactions_df = pd.concat([rent_df[common_cols], sale_df[common_cols]])

In [6]:
transactions_df.sample(5)

,property_id,agent_id,owner_id,client_id,transaction_date,transaction_amount,commission_payment_date,commission_amount,transaction_type
533,2430,196,1054,732,2022-05-28,175988.09,2022-08-24,5279.64,sale
453,880,128,524,1952,2022-10-04,80600.00,2024-05-20,8320.00,rent
1333,2069,147,834,1899,2024-09-30,113940.00,2024-10-15,11394.00,rent
2636,360,140,1703,804,2024-09-15,39300.00,2024-11-21,1965.00,rent
500,1717,218,1346,1476,2022-01-30,233882.37,2023-10-23,4677.65,sale


In [7]:
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4424 entries, 0 to 1644
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   property_id              4424 non-null   int64         
 1   agent_id                 4424 non-null   int64         
 2   owner_id                 4424 non-null   int64         
 3   client_id                4424 non-null   int64         
 4   transaction_date         4424 non-null   datetime64[ns]
 5   transaction_amount       4424 non-null   float64       
 6   commission_payment_date  4424 non-null   datetime64[ns]
 7   commission_amount        4424 non-null   float64       
 8   transaction_type         4424 non-null   object        
dtypes: datetime64[ns](2), float64(2), int64(4), object(1)
memory usage: 345.6+ KB


In [8]:
today_date = (transactions_df['commission_payment_date'].max()) + pd.Timedelta(days=1)  # تاريخ اليوم الافتراضي

# Owners Segmentation Using RFM Analysis

RFM score for each owner based on their historical transaction data
to measure value and engagement of owners


## Feature engineering: R,F,M

1. Recency --> today - nearest date (Sales + Rent)
2. Frequenc -->  count of transaction (Sales + Rent)
3. Monetary  --> total commission (Sales + Rent)

In [9]:
owner_agg_df = transactions_df.groupby("owner_id").agg(
                                                        last_date =("transaction_date","max"),
                                                        Frequency =("owner_id","count"),
                                                        Monetary =("commission_amount","sum")
).reset_index()

In [10]:
owner_agg_df['Recency'] = (today_date - owner_agg_df['last_date']).dt.days

In [11]:
owner_agg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1447 entries, 0 to 1446
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   owner_id   1447 non-null   int64         
 1   last_date  1447 non-null   datetime64[ns]
 2   Frequency  1447 non-null   int64         
 3   Monetary   1447 non-null   float64       
 4   Recency    1447 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 56.7 KB


In [12]:
owner_agg_df.head()

,owner_id,last_date,Frequency,Monetary,Recency
0,1,2024-09-10,5,13613.32,113
1,2,2024-12-31,7,68636.26,1
2,3,2024-12-31,5,43235.50,1
3,4,2024-12-31,5,35796.12,1
4,5,2024-02-19,1,14119.14,317


In [13]:
owner_agg_df.describe()

,owner_id,last_date,Frequency,Monetary,Recency
count,1447.000000,1447,1447.000000,1447.000000,1447.000000
mean,877.626123,2024-07-03 15:03:36.447823104,3.057360,31663.980171,181.372495
min,1.000000,2020-06-07 00:00:00,1.000000,438.000000,1.000000
25%,438.500000,2024-04-16 00:00:00,1.000000,11125.605000,1.000000
50%,885.000000,2024-12-10 00:00:00,3.000000,22752.000000,22.000000
75%,1313.500000,2024-12-31 00:00:00,4.000000,43899.265000,260.000000
max,1755.000000,2024-12-31 00:00:00,12.000000,242332.380000,1669.000000
std,507.570847,NaN,2.019312,28153.678105,291.206020


In [14]:
# Recency: most owners are very recent; fewer bins prevent clustering in a single bin -->  3 bins
# Frequency: 4 bins to provide a balanced distribution of owners based on transaction count, separating low and high activity
# Monetary: 4 bins to clearly classify profitability, covering a wide range of low and high values


## RFM Scoring 

Discretization (numerical -->categories"range")

In [30]:
# Recency score (high=old = 1 , low=new = 3)
bins_Recency   = [0, 30, 112, owner_agg_df['Recency'].max()]
labels_Recency = [3, 2, 1]

owner_agg_df['R_Score'] = pd.cut(
    owner_agg_df['Recency'],
    bins=bins_Recency,
    labels=labels_Recency,
    include_lowest=True
)

# Frequency score (low = 1, high= 4)
owner_agg_df['F_Score'] = pd.qcut(owner_agg_df['Frequency'].rank(method='first'), 4, labels=range(1,5))

# Monetary score (low = 1, high= 4)
owner_agg_df['M_Score'] = pd.qcut(owner_agg_df['Monetary'], 4, labels=range(1,5))

Combine R, F, M scores to form a single RFM score "string"

In [16]:
owner_agg_df['RFM_Score'] =( owner_agg_df['R_Score'].astype(str) + 
                             owner_agg_df['F_Score'].astype(str) + 
                             owner_agg_df['M_Score'].astype(str)
)

In [17]:
owner_agg_df.head()

,owner_id,last_date,Frequency,Monetary,Recency,R_Score,F_Score,M_Score,RFM_Score
0,1,2024-09-10,5,13613.32,113,1,4,2,142
1,2,2024-12-31,7,68636.26,1,3,4,4,344
2,3,2024-12-31,5,43235.50,1,3,4,3,343
3,4,2024-12-31,5,35796.12,1,3,4,3,343
4,5,2024-02-19,1,14119.14,317,1,1,2,112


RFM segmentaion:
- Champions: high in all three or at least very recent + (freq or monetary high) (RFM)
- Loyal Owners: frequent and high monetary but not the most recent (F&M) (R1/2)
- High Value – Low Activity: high monetary only (M) (F1/2,R3-1)
- Emerging Owne: recent and good spend but medium frequency (R) (F1-4, M2-4)
- At Risk: very old regardless of frequency or monetary (FM) (R=1)
- Dormant Owner: everyone else (mid-range)

In [18]:
def rfm_owner_segment(row):
    R, F, M = row['R_Score'], row['F_Score'], row['M_Score']

    if R == 3 and F >= 3 and M >= 3:
        return 'Champion'

    elif R >= 2 and F >= 3 and M >= 3:
        return 'Loyal Owner'

    elif R >= 2 and ((F >= 2 and 2 <= M <= 3) or (M >= 2 and 2 <= F <= 3)):
        return 'Emerging Owne'
    
    elif F < 3 and M >= 3:
        return 'High Value-Low Activity'
    
    elif R == 1 and F >= 2 and M >= 2:
        return 'At Risk'

    else:
        return 'Dormant Owner'

owner_agg_df['Segment'] = owner_agg_df.apply(rfm_owner_segment, axis=1)

In [19]:
owner_agg_df.head()

,owner_id,last_date,Frequency,Monetary,Recency,R_Score,F_Score,M_Score,RFM_Score,Segment
0,1,2024-09-10,5,13613.32,113,1,4,2,142,At Risk
1,2,2024-12-31,7,68636.26,1,3,4,4,344,Champion
2,3,2024-12-31,5,43235.50,1,3,4,3,343,Champion
3,4,2024-12-31,5,35796.12,1,3,4,3,343,Champion
4,5,2024-02-19,1,14119.14,317,1,1,2,112,Dormant Owner


## visualization

In [20]:
fig_bubble = px.scatter(owner_agg_df,
                        x='Recency', 
                        y='Monetary', 
                        size='Frequency',  
                        color='Segment', 
                        hover_data=['owner_id'], 
                        title='RFM Segmentation Scatter Plot')
fig_bubble.update_layout(xaxis_type="log")
fig_bubble.update_layout(yaxis_type="log")
fig_bubble.show()

In [ ]:
"""

Larger dots represent owners with higher transaction frequency.

Champions show high monetary value, frequent transactions, and recent activity, clustering in the top-right of the plot.

Loyal Owners have high frequency and spending but are less recent, clustering in the top-middle.

At Risk owners display high monetary value and frequency but long inactivity, clustering in the top-left.

High Value-Low Activity owners indicate high spending with few transactions.

Dormant Owners show long inactivity or low spending, clustering in the lower area of the plot.

Emerging Owners demonstrate recent activity with moderate to high frequency or spending.

"""

'\n\nLarger dots represent owners with higher transaction frequency.\n\nChampions show high monetary value, frequent transactions, and recent activity, clustering in the top-right of the plot.\n\nLoyal Owners have high frequency and spending but are less recent, clustering in the top-middle.\n\nAt Risk owners display high monetary value and frequency but long inactivity, clustering in the top-left.\n\nHigh Value-Low Activity owners indicate high spending with few transactions.\n\nDormant Owners show long inactivity or low spending, clustering in the lower area of the plot.\n\nEmerging Owners demonstrate recent activity with moderate to high frequency or spending.\n\n'

In [22]:
fig_counts = px.histogram(owner_agg_df, 
                          x='Segment', 
                          title='Number of Owners per Segment', 
                          color='Segment', 
                          text_auto=True)
fig_counts.show()

In [ ]:
"""
The largest segment is Dormant Owners, followed closely by Champions                                                --> Activate Dormant owners through market insights and management packages, while retaining Champions with premium long-term contracts.
Emerging owners form a moderate portion, showing potential for growth if nurtured properly.                         --> Support them with property pricing guidance and marketing strategies to move them toward Champions.
Meanwhile, At Risk and High Value Active owners appear in smaller numbers                                           --> Requiring targeted retention efforts. 
Loyal owners have the lowest share, which highlights a challenge in maintaining long-term commitment among clients. --> Encourage advocacy by offering referral rewards and maintenance discounts
"""

'\nThe majority of owners fall into the Dormant and At Risk segments, indicating a high risk of churn.\n\nChampions and Loyal Owners represent a smaller but high-value share of the owner base.\n\nGrowth opportunities lie in converting Emerging Owners into Loyal Owners through targeted engagement.\n'

In [31]:
owner_agg_df[['owner_id', 'R_Score', 'F_Score', 'M_Score', 'Segment']].to_csv(
    "owner_segments.csv", index=False)

# Clients Segmentation Using RFM Analysis

RFM score for each client based on their historical transaction data.



## Feature engineering: R,F,M

1. Recency: How recently a client made a purchase / rent
2. Frequency: How often they made purchases / rent
3. Monetary: How much money they spent

In [32]:
client_agg_df = transactions_df.groupby("client_id").agg(
                                                        last_date =("transaction_date","max"),
                                                        Frequency =("client_id","count"),
                                                        Monetary =("transaction_amount","sum")
).reset_index()

In [33]:
client_agg_df['Recency'] = (today_date - client_agg_df['last_date']).dt.days

In [34]:
client_agg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1827 entries, 0 to 1826
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   client_id  1827 non-null   int64         
 1   last_date  1827 non-null   datetime64[ns]
 2   Frequency  1827 non-null   int64         
 3   Monetary   1827 non-null   float64       
 4   Recency    1827 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 71.5 KB


In [35]:
client_agg_df.head()

,client_id,last_date,Frequency,Monetary,Recency
0,1,2024-12-29,1,47200.00,3
1,2,2024-05-23,1,256291.77,223
2,3,2024-12-31,1,130260.00,1
3,4,2024-02-02,1,458279.66,334
4,5,2024-05-09,1,142100.00,237


In [36]:
client_agg_df.describe()

,client_id,last_date,Frequency,Monetary,Recency
count,1827.000000,1827,1827.000000,1.827000e+03,1827.000000
mean,1028.672687,2024-08-09 20:01:58.226600960,2.421456,5.656091e+05,144.165298
min,1.000000,2020-05-01 00:00:00,1.000000,7.080000e+03,1.000000
25%,521.500000,2024-07-24 00:00:00,1.000000,1.668300e+05,1.000000
50%,1025.000000,2024-12-25 00:00:00,2.000000,3.535561e+05,7.000000
75%,1538.500000,2024-12-31 00:00:00,3.000000,7.666638e+05,161.000000
max,2060.000000,2024-12-31 00:00:00,10.000000,4.814742e+06,1706.000000
std,590.926779,NaN,1.324526,5.715645e+05,267.794900


## RFM Scoring 

Discretization (numerical -->categories"range")

In [37]:
# Recency score (high=old = 1 , low=new = 3)
bins_Recency   = [0, 30, 224, client_agg_df['Recency'].max()]
labels_Recency = [3, 2, 1]

client_agg_df['R_Score'] = pd.cut(
    client_agg_df['Recency'],
    bins=bins_Recency,
    labels=labels_Recency,
    include_lowest=True
)

# Frequency score (low = 1, high= 4)
client_agg_df['F_Score'] = pd.qcut(client_agg_df['Frequency'].rank(method='first'), 4, labels=range(1,5))

# Monetary score (low = 1, high= 4)
client_agg_df['M_Score'] = pd.qcut(client_agg_df['Monetary'], 4, labels=range(1,5))

Combine R, F, M scores to form a single RFM score "string"

In [38]:
client_agg_df['RFM_Score'] =( client_agg_df['R_Score'].astype(str) + 
                             client_agg_df['F_Score'].astype(str) + 
                             client_agg_df['M_Score'].astype(str)
)

In [39]:
client_agg_df.head()

,client_id,last_date,Frequency,Monetary,Recency,R_Score,F_Score,M_Score,RFM_Score
0,1,2024-12-29,1,47200.00,3,3,1,1,311
1,2,2024-05-23,1,256291.77,223,2,1,2,212
2,3,2024-12-31,1,130260.00,1,3,1,1,311
3,4,2024-02-02,1,458279.66,334,1,1,3,113
4,5,2024-05-09,1,142100.00,237,1,1,1,111


RFM segmentaion:
- VIP Clients: high in all three or at least very recent + (freq or monetary high) (RFM)
- Regular High-Value: frequent and high monetary but not the most recent (F&M) (R1/2)
- Big Spenders: high monetary only (M) (F1/2,R3-1)
- Potential Loyalists: recent and good spend but medium frequency (R) (F1-4, M2-4)
- At Risk: very old regardless of frequency or monetary (FM) (R=1)
- Mid-Tier: everyone else (mid-range)

In [40]:
def rfm_client_segment(row):
    R, F, M = row['R_Score'], row['F_Score'], row['M_Score']

    if R == 3 and F >= 3 and M >= 3:
        return 'VIP Clients'

    elif R >= 2 and F >= 3 and M >= 3:
        return 'Regular High-Value'

    elif R >= 2 and ((F >= 2 and 2 <= M <= 3) or (M >= 2 and 2 <= F <= 3)):
        return 'Potential Loyalist'
    
    elif F < 3 and M >= 3:
        return 'Big Spender'

    elif R == 1 and F >= 2 and M >= 2:
        return 'At Risk'

    else:
        return 'Mid-Tier'

client_agg_df['Segment'] = client_agg_df.apply(rfm_client_segment, axis=1)

In [41]:
client_agg_df.head()

,client_id,last_date,Frequency,Monetary,Recency,R_Score,F_Score,M_Score,RFM_Score,Segment
0,1,2024-12-29,1,47200.00,3,3,1,1,311,Mid-Tier
1,2,2024-05-23,1,256291.77,223,2,1,2,212,Mid-Tier
2,3,2024-12-31,1,130260.00,1,3,1,1,311,Mid-Tier
3,4,2024-02-02,1,458279.66,334,1,1,3,113,Big Spender
4,5,2024-05-09,1,142100.00,237,1,1,1,111,Mid-Tier


## visualization

In [42]:
fig_bubble = px.scatter(client_agg_df,
                        x='Recency', 
                        y='Monetary', 
                        size='Frequency',  
                        color='Segment', 
                        hover_data=['client_id'], 
                        title='RFM Segmentation Scatter Plot')
fig_bubble.update_layout(xaxis_type="log")
fig_bubble.update_layout(yaxis_type="log")
fig_bubble.show()

In [43]:
fig_counts = px.histogram(client_agg_df, 
                          x='Segment', 
                          title='Number of Client per Segment', 
                          color='Segment', 
                          text_auto=True)
fig_counts.show()

In [ ]:
"""
The majority of clients fall into the Mid-Tier segment        --> Focus on converting them into High-Value clients through bundle offers, personalized follow-ups, and targeted promotions.
VIPs and Potential Loyalists follow with close proportions    --> Retain them with exclusive services, priority access to new listings, and loyalty programs.
Regular High-Value and Big Spenders represent smaller shares  --> Provide premium property deals, and dedicated account management.
At Risk clients form the lowest segment                       --> Re-engage them with surveys, discounts, or flexible payment options to reduce churn.
"""

'\nThe majority of clients fall into the Mid-Tier and At Risk segments, indicating a high risk of churn.\n\nVIP and Regular High-Value clients represent a smaller but highly profitable portion of the customer base.\n\nRecommended Actions:\n\nRetention --> Launch personalized campaigns to reactivate Mid-Tier and At Risk clients before they churn.\n\nGrowth --> Focus on nurturing Potential Loyalist clients with targeted offers and engagement programs to convert them into High-Value clients.\n'

In [46]:
client_agg_df[['client_id', 'R_Score', 'F_Score', 'M_Score', 'Segment']].to_csv(
    "client_segments.csv", index=False)